<a href="https://colab.research.google.com/github/Sandell0/Colab-experiments/blob/master/Telegram_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install transformers
%pip install datasets

In [ ]:
import torch
import tqdm
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BatchEncoding,
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
    TrainerCallback
)

In [ ]:
torch.cuda.empty_cache()

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")
tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    tokenized_text = tokenizer(
        examples["text"],
        truncation=True,
        padding=True,
        max_length=512,
    )
    examples["input_ids"] = tokenized_text["input_ids"].copy()
    examples["attention_mask"] = tokenized_text["attention_mask"].copy()
    examples["labels"] = tokenized_text["input_ids"].copy()

    return examples

In [ ]:
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/Github/weights/")

prompts = [
    "Я",
]


# encode prompt
encoded_prompts = tokenizer(prompts, return_tensors="pt")
encoded_prompts = encoded_prompts.to('cuda:0')
model = model.to('cuda:0')

# generate text
outputs = model.generate(
    input_ids=encoded_prompts["input_ids"],
    attention_mask=encoded_prompts["attention_mask"],
    max_length=512,
)

print(tokenizer.decode(outputs[0]).replace('\n',''))

In [ ]:

data = load_dataset("text", data_files="dataset.csv")
train_test = data["train"].train_test_split(test_size=0.05, shuffle=False)

for key in ["train", "test"]:
    train_test[key] = train_test[key].map(tokenize_function)

model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")

training_args = TrainingArguments(
    output_dir="./eu-chatbot",  # The output directory
    overwrite_output_dir=True,  # overwrite the content of the output directory
    num_train_epochs=3,  # number of training epochs
    per_device_train_batch_size=1,  # batch size for training
    per_device_eval_batch_size=1,  # batch size for evaluation
    eval_steps=1000,  # 400,  # Number of update steps between two evaluations.
    save_steps=1000,  # after # steps model is saved
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    logging_strategy="steps",
    logging_steps=200,
    save_total_limit=10,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    learning_rate=5e-6,
)

class MyCallback(TrainerCallback):
    def on_log(self, args, state, control, **kwargs):
      prompts = [
          "",
      ]


      # encode prompt
      encoded_prompts = tokenizer(prompts, return_tensors="pt")

      encoded_prompts = encoded_prompts.to('cuda:0')

      # generate text
      outputs = model.generate(
          input_ids=encoded_prompts["input_ids"],
          attention_mask=encoded_prompts["attention_mask"],
          max_length=512,
      )

      print(tokenizer.decode(outputs[0]).replace('\n',''))



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test["train"],
    eval_dataset=train_test["test"],
    callbacks=[EarlyStoppingCallback(early_stopping_patience=30), MyCallback],
)

trainer.train()
trainer.save_model()